In [1]:
from pyspark.sql import SparkSession 
spark=(SparkSession 
          .builder
          .appName("MyApp")
          .getOrCreate())

25/05/11 03:31:49 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/05/11 03:31:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/11 03:31:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [4]:
df1=spark.read.csv("/home/daman/Downloads/archive/bookdata.csv",header=True)
df1.createOrReplaceTempView('bookdata')
df1.limit(2).show()


+------------+--------+--------------------+-----------+-----+--------------+--------------+------------+-----+---+--------------------+----------------+--------------------+
|availability|category|         description|num_reviews|price|price_excl_tax|price_incl_tax|product_type|stars|tax|               title|             upc|                 url|
+------------+--------+--------------------+-----------+-----+--------------+--------------+------------+-----+---+--------------------+----------------+--------------------+
|          19| default|Since her assault...|          0|13.99|         13.99|         13.99|       books|    2|0.0|Since her assault...|0312262ecafa5a40|https://books.tos...|
|          20| fiction|Dans une France a...|          0| 50.1|          50.1|          50.1|       books|    1|0.0|Dans une France a...|6957f44c3847a760|https://books.tos...|
+------------+--------+--------------------+-----------+-----+--------------+--------------+------------+-----+---+----------

In [5]:
df2=spark.read.json("/home/daman/Downloads/archive/Books.json")
df2.createOrReplaceTempView('books')
df2.limit(2).show()


+--------------------+------------+--------------+--------------------+----+-----------+-----+--------------+--------------+------------+-----+----+--------------------+----------------+--------------------+
|     _corrupt_record|availability|      category|         description|  id|num_reviews|price|price_excl_tax|price_incl_tax|product_type|stars| tax|               title|             upc|                 url|
+--------------------+------------+--------------+--------------------+----+-----------+-----+--------------+--------------+------------+-----+----+--------------------+----------------+--------------------+
|[{"id":1, "url":"...|        NULL|          NULL|                NULL|NULL|       NULL| NULL|          NULL|          NULL|        NULL| NULL|NULL|                NULL|            NULL|                NULL|
|                NULL|          19|sequential art|('Scott Pilgrim\'...|   2|          0| 52.0|          52.0|          52.0|       books|    5| 0.0|Scott Pilgrim's l...

In [6]:
display(spark.sql("""
with temp_books as (select * from books where _corrupt_record is Null)
select
bookdata.*,
temp_books.category
from temp_books
left join bookdata
on temp_books.upc = bookdata.upc
""").toPandas())

,availability,category,description,num_reviews,price,price_excl_tax,price_incl_tax,product_type,stars,tax,title,upc,url,category
0,None,None,None,None,None,None,None,None,None,None,None,None,None,sequential art
1,19,young adult,Aaron Ledbetter’s future had been planned out ...,0,17.46,17.46,17.46,books,5,0.0,Aaron Ledbetter’s future had been planned out ...,ce6396b0f23f6ecc,https://books.toscrape.com/catalogue/set-me-fr...,young adult
2,20,history,From a renowned historian comes a groundbreaki...,0,54.23,54.23,54.23,books,5,0.0,From a renowned historian comes a groundbreaki...,4165285e1663650f,https://books.toscrape.com/catalogue/sapiens-a...,history
3,19,poetry,This book is an important and complete collect...,0,20.66,20.66,20.66,books,4,0.0,This book is an important and complete collect...,30a7f60cd76ca58c,https://books.toscrape.com/catalogue/shakespea...,poetry
4,20,mystery,"WICKED above her hipbone, GIRL across her hear...",0,47.82,47.82,47.82,books,4,0.0,"WICKED above her hipbone, GIRL across her hear...",e00eb4fd7b871a48,https://books.toscrape.com/catalogue/sharp-obj...,mystery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,1,mystery,"James Patterson, bestselling author of the Ale...",0,53.98,53.98,53.98,books,1,0.0,"James Patterson, bestselling author of the Ale...",f684a82adc49f011,https://books.toscrape.com/catalogue/1st-to-di...,mystery
993,None,None,None,None,None,None,None,None,None,None,None,None,None,religion
994,1,historical fiction,"In England’s Regency era, manners and elegance...",0,16.97,16.97,16.97,books,5,0.0,"In England’s Regency era, manners and elegance...",19fec36a1dfb4c16,https://books.toscrape.com/catalogue/a-spys-de...,historical fiction
995,1,default,Mary Shelley began writing Frankenstein when s...,0,38.0,38.0,38.0,books,2,0.0,Mary Shelley began writing Frankenstein when s...,a492f49a3e2b6a71,https://books.toscrape.com/catalogue/frankenst...,default
